# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import re
import pickle
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, HashingVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer,confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.svm import SVC
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.externals import joblib


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
#Read the data from DisasterResponse table created in the ETL pipeline
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table( 'messages' , engine)
#Define feature
X = df['message']
#Define target classes
y = df[df.columns[4:]]
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X = df['message']
y = df.loc[:, 'related':'direct_report']

In [6]:
y.columns.tolist()

['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'child_alone',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

In [7]:
y['related'] = y['related'].replace(2, 1)

In [8]:
y['related'].unique()

array([1, 0])

### 2. Write a tokenization function to process your text data

In [9]:
def tokenize(text):
    text = re.sub(r'[^\w\s]', '', text)
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = [lemmatizer.lemmatize(tok).lower().strip() for tok in tokens]

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [13]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [15]:
y_pred = pipeline.predict(X_test)
y_pred_df = pd.DataFrame(y_pred, columns=y_test.columns)

evaluation = {}
for column_true, column_pred in zip(y_test.columns, y_pred_df.columns):
    precision = precision_score(y_test[column_true], y_pred_df[column_pred])
    recall = recall_score(y_test[column_true], y_pred_df[column_pred])
    f1 = f1_score(y_test[column_true], y_pred_df[column_pred])
    evaluation[column_true] = [precision, recall, f1]

evaluation_df = pd.DataFrame(evaluation)

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [21]:
evaluation_df.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0.831139,0.816901,0.0,0.744285,0.552239,0.571429,0.583333,0.333333,0.720000,0.0,...,0.0,0.142857,0.838710,0.853333,0.792079,0.0,0.887955,0.782609,0.818182,0.809917
1,0.927706,0.355828,0.0,0.526118,0.076923,0.080537,0.044304,0.009901,0.082949,0.0,...,0.0,0.003436,0.523965,0.241966,0.414508,0.0,0.496865,0.130435,0.053097,0.302703
2,0.876771,0.495726,0.0,0.616469,0.135036,0.141176,0.082353,0.019231,0.148760,0.0,...,0.0,0.006711,0.644988,0.377025,0.544218,0.0,0.637186,0.223602,0.099723,0.440697


In [22]:
columnas = y_test.columns
for i, columna in enumerate(columnas):
    print(f"Precision, Recall, F1 Score for {columna}")
    report = classification_report(y_test.iloc[:, i], y_pred[:, i])
    print(report)

Precision, Recall, F1 Score for related
             precision    recall  f1-score   support

          0       0.61      0.38      0.46      1519
          1       0.83      0.93      0.88      5035

avg / total       0.78      0.80      0.78      6554

Precision, Recall, F1 Score for request
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      5413
          1       0.82      0.36      0.50      1141

avg / total       0.87      0.87      0.85      6554

Precision, Recall, F1 Score for offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6531
          1       0.00      0.00      0.00        23

avg / total       0.99      1.00      0.99      6554

Precision, Recall, F1 Score for aid_related
             precision    recall  f1-score   support

          0       0.73      0.88      0.80      3893
          1       0.74      0.53      0.62      2661

avg / total       0.74      0.73

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [27]:
parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'vect__max_df': (0.5, 0.75, 1.0),
        'tfidf__use_idf': (True, False)
    }

cv = GridSearchCV(pipeline, param_grid=parameters)

In [28]:
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [30]:
print("Accuracy")
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

for i, column in enumerate(y_test.columns[:35]):
    print(f"Precision, Recall, F1 Score for {column}")
    report = classification_report(y_test.iloc[:, i], y_pred[:, i])
    print(report)

Accuracy
0.246376811594
Precision, Recall, F1 Score for related
             precision    recall  f1-score   support

          0       0.61      0.37      0.46      1266
          1       0.82      0.92      0.87      3978

avg / total       0.77      0.79      0.77      5244

Precision, Recall, F1 Score for request
             precision    recall  f1-score   support

          0       0.89      0.99      0.93      4349
          1       0.85      0.38      0.53       895

avg / total       0.88      0.88      0.86      5244

Precision, Recall, F1 Score for offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5218
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      5244

Precision, Recall, F1 Score for aid_related
             precision    recall  f1-score   support

          0       0.72      0.89      0.80      3113
          1       0.76      0.49      0.60      2131

avg / to

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [33]:
from sklearn.pipeline import make_pipeline
pipeline = make_pipeline(
    CountVectorizer(tokenizer=tokenize),
    TfidfTransformer(),
    MultiOutputClassifier(AdaBoostClassifier())
)

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

In [36]:
report = classification_report(y_test, y_pred, target_names=y_test.columns[:35])
print(report)

                        precision    recall  f1-score   support

               related       0.82      0.95      0.88      3978
               request       0.77      0.52      0.62       895
                 offer       0.25      0.08      0.12        26
           aid_related       0.75      0.59      0.66      2131
          medical_help       0.58      0.27      0.37       422
      medical_products       0.68      0.30      0.42       270
     search_and_rescue       0.55      0.20      0.30       127
              security       0.10      0.02      0.04        88
              military       0.54      0.33      0.41       155
           child_alone       0.00      0.00      0.00         0
                 water       0.74      0.61      0.67       339
                  food       0.83      0.68      0.75       595
               shelter       0.79      0.55      0.65       470
              clothing       0.79      0.42      0.55        73
                 money       0.46      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 36, does not match size of target_names, 35
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 9. Export your model as a pickle file

In [37]:
import pickle
filename = 'finalized_model.pkl'
pickle.dump(pipeline, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.